## 导入相关包，创建相机实例

In [ ]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg
from jetbot import Robot
import PID
camera = Camera.instance(width=720, height=720)

## 创建相关控制变量

In [ ]:
global color_x, color_y, color_radius
color_x = color_y = color_radius = 0
global target_valuex
target_valuex = 2100
global target_valuey
target_valuey = 2048

## 创建存储HSV色域颜色分类数据的数组

In [ ]:
import numpy as np
global color_lower
color_lower=np.array([156,43,46])
global color_upperv
color_upper = np.array([180, 255, 255])

## 创建PID控制实例

In [ ]:
xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

## 创建云台总线舵机实例

In [ ]:
from servoserial import ServoSerial
servo_device = ServoSerial() 

## 创建显示控件

In [ ]:
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
color_image = widgets.Image(format='jpeg', width=300, height=300)
display(color_image)

## 云台运动主进程

In [ ]:
from time import sleep
robot = Robot()
robot.forward(1)#Jetbot前进

In [ ]:
from RGB_Lib import Programing_RGB
RGB = Programing_RGB()

# 定义颜色范围
red_lower = np.array([120, 77, 112])
red_upper = np.array([180, 168, 248])
green_lower = np.array([52, 110, 48])
green_upper = np.array([180, 255, 163])
yellow_lower = np.array([17, 79, 100])
yellow_upper = np.array([61, 152, 255])

# 颜色处理函数
def process_color(frame, color_lower, color_upper):
    frame_ = cv2.GaussianBlur(frame, (5,5), 0)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, color_lower, color_upper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    mask = cv2.GaussianBlur(mask, (3,3), 0)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    return cnts

while True:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300))
    
    # 检测红色
    red_cnts = process_color(frame, red_lower, red_upper)
    if len(red_cnts) > 0:
        cnt = max(red_cnts, key=cv2.contourArea)
        (color_x, color_y), color_radius = cv2.minEnclosingCircle(cnt)
        if color_radius > 10:
            cv2.circle(frame, (int(color_x), int(color_y)), int(color_radius), (0, 0, 255), 2)
            # TODO: 红色反应逻辑 - 停止
            # servo_device.Servo_serial_double_control(1, 停止位置, 2, 停止位置)
            print("检测到红色 - 停止")
            sleep(0.5)
            RGB.Set_All_RGB(0xFF, 0x00, 0x00)
            robot.stop()  #停止Jetbot
            
    # 检测绿色
    green_cnts = process_color(frame, green_lower, green_upper)
    if len(green_cnts) > 0:
        cnt = max(green_cnts, key=cv2.contourArea)
        (color_x, color_y), color_radius = cv2.minEnclosingCircle(cnt)
        if color_radius > 10:
            cv2.circle(frame, (int(color_x), int(color_y)), int(color_radius), (0, 255, 0), 2)
            # TODO: 绿色反应逻辑 - 前进
            # servo_device.Servo_serial_double_control(1, 前进位置, 2, 前进位置)
            print("检测到绿色 - 前进")
            sleep(0.5)
            robot.forward(1)  #Jetbot前进
            RGB.Set_All_RGB(0x00, 0xFF, 0x00)
    
    # 检测黄色
    yellow_cnts = process_color(frame, yellow_lower, yellow_upper)
    if len(yellow_cnts) > 0:
        cnt = max(yellow_cnts, key=cv2.contourArea)
        (color_x, color_y), color_radius = cv2.minEnclosingCircle(cnt)
        if color_radius > 10:
            cv2.circle(frame, (int(color_x), int(color_y)), int(color_radius), (0, 255, 255), 2)
            # TODO: 黄色反应逻辑 - 减速
            print("检测到黄色 - 减速")
            sleep(0.5)
            robot.left_motor.value = 0.7
            robot.right_motor.value = 0.7
            RGB.Set_All_RGB(0xFF, 0xFF, 0x00)
    
    # 实时传回图像数据进行显示
    color_image.value = bgr8_to_jpeg(frame)